In [1]:
import numpy as np
import numpy.linalg as la
from typing import Callable
import matplotlib.pyplot as plt
import sympy as sp
import pandas as pd
pd.set_option("display.precision", 8)


# Newton Non-Linear equation 

$$f(\mathbf x) = \begin{bmatrix}3x_1 - \cos{x_2 x_3} - 0.5\\x_1^2 - 81{(x_2 + 0.1)}^2 + \sin{x_3} + 1.06\\e^{-x_1x_2} + 20x_3 + \frac{10\pi - 3}{3} \end{bmatrix}$$

In [2]:
# x0 = np.array([0.1, 0.1, 0.1])
f0 = lambda x: np.array([3 * x[0] - np.cos(x[1] * x[2]) - 0.5,
                         x[0]**2 - 81.0 * (x[1] + 0.1)**2 + np.sin(x[2]) + 1.06,
                         np.e**(-x[0] * x[1]) + 20 * x[2] + (10 * np.pi - 3.0) / 3.0])

df0 = lambda x: np.array([[3, x[2] * np.sin(x[1] * x[2]), x[1] * np.sin(x[1] * x[2])],
                          [2 * x[0], -162 * (x[1] + 0.1), np.cos(x[2])],
                          [-x[1] * np.e**(-x[0] * x[1]), -x[0] * np.e**(-x[0] * x[1]), 20]])


In [3]:
def newton_method(f: Callable, df: Callable, x0: np.ndarray, MAX_ITER=100, tol=1e-8):
    for k in range(1, MAX_ITER):
        f_val = f(x0)
        df_val = df(x0)
        dx = np.linalg.solve(df_val, f_val)
        x = x0 - dx
        abs_err = la.norm(x - x0, np.inf)

        print(f"{k:02d} {x} {abs_err:0.8f}")
        if abs_err < tol:
            return x
        x0 = x.copy()
    print("Too many iterations")
    return None


In [4]:
x0 = np.array([1.0, 1.0, 1.0])
np.set_printoptions(precision=8, suppress=True)
print("k  x[k]\t\t\t\t\t error absolute")
newton_method(f0, df0, x0);


k  x[k]					 error absolute
01 [ 0.91968721  0.46082246 -0.50338764] 1.50338764
02 [ 0.50100049  0.18743348 -0.52086923] 0.41868673
03 [ 0.50054294  0.06115345 -0.52200096] 0.12628002
04 [ 0.50010444  0.01161711 -0.52329515] 0.04953635
05 [ 0.50000551  0.00060562 -0.52358294] 0.01101149
06 [ 0.50000002  0.00000183 -0.52359873] 0.00060379
07 [ 0.5         0.         -0.52359878] 0.00000183
08 [ 0.5        -0.         -0.52359878] 0.00000000


In [5]:
A = np.array([[5, 0.5, -1],
              [0.5, 2, 0.],
              [-1., 0, 8.]])

b = np.array([2, 3, -1.0])

c = 10

f = lambda x: np.dot(x, A @ x) + np.dot(b, x) + c
df = lambda x: 2 * A @ x + b
d2f = lambda x: 2 * A


# Newton Optimization

Iteración
$$x_{k+1}=x_k - {[\nabla ^2 f(x_k)]}^{-1} \nabla  f(x_k)$$

In [6]:
def newton_optimization(f: Callable, df: Callable, ddf: Callable, x: np.ndarray, MAX_ITER=100, tol=1e-7):
    print(f"{0:02d} {x}")
    for k in range(1, MAX_ITER):
        # calculate gradient and hessian
        gradient = df(x)
        hessian = ddf(x)
        # update step
        x = x - la.inv(hessian) @ gradient
        grad_err = la.norm(df(x), np.inf)
        # print step
        print(f"{k:02d} {x} {grad_err:0.8f}")
        # stop criterion |∇f| = 0
        if grad_err < tol:
            return x
    print("Too many iterations")
    return None


# Laboratorio Semana 11: Método de Newton

Aplicando el método de Newton Minimizar las siguientes funciones

##### a) $f(\mathbf x) = 7x_1^2 + 2x_1x_2 + x_2^2$

In [8]:
fa = lambda x: 7 * x[0]**2 + 2 * x[0] * x[1] + x[1]**2
dfa = lambda x: np.array([14 * x[0] + 2 * x[1], 2 * x[0] + 2 * x[1]])
ddfa = lambda x: np.array([[14, 2],
                           [2, 2.]])

x0 = np.array([-2, 2.])
print("k  x[k]\t\terror |∇f|=0")
newton_optimization(fa, dfa, ddfa, x0);


k  x[k]		error |∇f|=0
00 [-2.  2.]
01 [-0.  0.] 0.00000000


##### b) $f(\mathbf x)=x_{1}^{2} + x_{1} x_{2} + 2 x_{1} x_{3} + 4 x_{2}^{2} + x_{2} x_{3} + 4 x_{3}^{2}$

In [9]:
fb = lambda x: x[0]**2 + x[0] * x[1] + 2 * x[0] * x[2] + 4 * x[1]**2 + x[1] * x[2] + 4 * x[2]**2
dfb = lambda x: np.array([2 * x[0] + x[1] + 2 * x[2], x[0] + 8 * x[1] + x[2], 2 * x[0] + x[1] + 8 * x[2]])
ddfb = lambda x: np.array([[2, 1, 2],
                           [1, 8, 1],
                           [2, 1, 8.]])
                           
x0 = np.array([1.0, 1.0, 1.0])
print("k  x[k]\t\terror |∇f|=0")
newton_optimization(fb, dfb, ddfb, x0);


k  x[k]		error |∇f|=0
00 [1. 1. 1.]
01 [0. 0. 0.] 0.00000000


##### c) $f(\mathbf x)=4 \left(x_{1} - 2\right)^{4} + 2 \left(x_{1} - 2 x_{2}\right)^{2}$

In [10]:
fc = lambda x: 4 * (x[0] - 2)**4 + 2 * (x[0] - 2 * x[1])**2
dfc = lambda x: np.array([4 * x[0] - 8 * x[1] + 16 * (x[0] - 2)**3, -8 * x[0] + 16 * x[1]])
ddfc = lambda x: np.array([[48 * (x[0] - 2)**2 + 4, -8],
                           [-8, 16]])
x0 = np.array([1.0, 1.2])
print("k  x[k]\t\t\terror |∇f|=0")
newton_optimization(fc, dfc, ddfc, x0);


k  x[k]			error |∇f|=0
00 [1.  1.2]
01 [1.33333333 0.66666667] 4.74074074
02 [1.55555556 0.77777778] 1.40466392
03 [1.7037037  0.85185185] 0.41619672
04 [1.80246914 0.90123457] 0.12331755
05 [1.86831276 0.93415638] 0.03653853
06 [1.9122085  0.95610425] 0.01082623
07 [1.94147234 0.97073617] 0.00320777
08 [1.96098156 0.98049078] 0.00095045
09 [1.97398771 0.98699385] 0.00028162
10 [1.98265847 0.99132924] 0.00008344
11 [1.98843898 0.99421949] 0.00002472
12 [1.99229265 0.99614633] 0.00000733
13 [1.99486177 0.99743088] 0.00000217
14 [1.99657451 0.99828726] 0.00000064
15 [1.99771634 0.99885817] 0.00000019
16 [1.99847756 0.99923878] 0.00000006


##### d) $f(\mathbf x)=\left(x_{1} - 2\right)^{4} + \left(x_{1} - x_{2}\right)^{2}$


In [11]:
fd = lambda x: (x[0] - 2)**4 + (x[0] - x[1])**2
dfd = lambda x: np.array([2 * x[0] - 2 * x[1] + 4 * (x[0] - 2)**3, -2 * x[0] + 2 * x[1]])
ddfd = lambda x: np.array([[12 * (x[0] - 2)**2 + 2, -2], [-2, 2]])

x0 = np.array([1.0, 1.2])
print("k  x[k]\t\t\terror |∇f|=0")
newton_optimization(fd, dfd, ddfd, x0);


k  x[k]			error |∇f|=0
00 [1.  1.2]
01 [1.33333333 1.33333333] 1.18518519
02 [1.55555556 1.55555556] 0.35116598
03 [1.7037037 1.7037037] 0.10404918
04 [1.80246914 1.80246914] 0.03082939
05 [1.86831276 1.86831276] 0.00913463
06 [1.9122085 1.9122085] 0.00270656
07 [1.94147234 1.94147234] 0.00080194
08 [1.96098156 1.96098156] 0.00023761
09 [1.97398771 1.97398771] 0.00007040
10 [1.98265847 1.98265847] 0.00002086
11 [1.98843898 1.98843898] 0.00000618
12 [1.99229265 1.99229265] 0.00000183
13 [1.99486177 1.99486177] 0.00000054
14 [1.99657451 1.99657451] 0.00000016
15 [1.99771634 1.99771634] 0.00000005


##### e) $f(\mathbf x)=x_{1}^{4} + \left(x_{1} + x_{2}\right)^{2} + \left(e^{x_{2}} - 1\right)^{2}$


In [12]:
fe = lambda x: x[0]**4 + (x[0] + x[1])**2 + (np.exp(x[1]) - 1)**2
dfe = lambda x: np.array([4 * x[0]**3 + 2 * x[0] + 2 * x[1], 2 * x[0] + 2 * x[1] + 2 * (np.exp(x[1]) - 1) * np.exp(x[1])])
ddfe = lambda x: np.array([[12 * x[0]**2 + 2, 2],
                           [2, 2 * (np.exp(x[1]) - 1) * np.exp(x[1]) + 2 * np.exp(2 * x[1]) + 2]])

x0 = np.array([1.0, 1.2])
print("k  x[k]\t\t\terror |∇f|=0")
newton_optimization(fe, dfe, ddfe, x0);


k  x[k]			error |∇f|=0
00 [1.  1.2]
01 [0.46786411 0.72495126] 6.78176767
02 [0.03509768 0.32846156] 1.80715549
03 [-0.10836193  0.10933579] 0.25971898
04 [-0.02164563  0.01808097] 0.03002732
05 [-0.00055387  0.00051486] 0.00095250
06 [-0.0000004  0.0000004] 0.00000079
07 [-0.  0.] 0.00000000


##### f) $f(\mathbf x)=\frac{3 x_{1}^{2}}{2} + x_{1} x_{3} - x_{1} + 2 x_{2}^{2} + 2 x_{2} x_{3} - x_{2} + \frac{3 x_{3}^{2}}{2} - x_{3} + 3$


In [19]:
ff = lambda x: 3 * x[0]**2 / 2 + x[0] * x[2] - x[0] + 2 * x[1]**2 + 2 * x[1] * x[2] - x[1] + 3 * x[2]**2 / 2 - x[2] + 3
dff = lambda x: np.array([3 * x[0] + x[2] - 1, 4 * x[1] + 2 * x[2] - 1, x[0] + 2 * x[1] + 3 * x[2] - 1])
ddff = lambda x: np.array([[3, 0, 1],
                           [0, 4, 2],
                           [1, 2, 3]])

x0 = np.array([1.0, 1.0, 1.0])
print("k  x[k]\t\terror |∇f|=0")
newton_optimization(ff, dff, ddff, x0);


k  x[k]		error |∇f|=0
00 [1. 1. 1.]
01 [0.3 0.2 0.1] 0.00000000


##### g) $f(\mathbf x)=- 2 e^{- \left(x_{1} - 1\right)^{2} - \left(x_{2} - 1\right)^{2}}$

In [25]:
fg = lambda x: -2 * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2)
dfg = lambda x: np.array([-2 * (2 - 2 * x[0]) * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2), -2 * (2 - 2 * x[1]) * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2)])
ddfg = lambda x: np.array([[-2 * (2 - 2 * x[0])**2 * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2) + 4 * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2), -2 * (2 - 2 * x[0]) * (2 - 2 * x[1]) * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2)],
                           [-2 * (2 - 2 * x[0]) * (2 - 2 * x[1]) * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2), -2 * (2 - 2 * x[1])**2 * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2) + 4 * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2)]])

x0 = np.array([1.0, -1.0])
print("k  x[k]\t\terror |∇f|=0")
newton_optimization(fg, dfg, ddfg, x0);


k  x[k]		error |∇f|=0
00 [ 1. -1.]
01 [ 1.         -1.28571429] 0.04921697
02 [ 1.         -1.52761493] 0.01698768
03 [ 1.         -1.74222563] 0.00594778
04 [ 1.         -1.93754637] 0.00210107
05 [ 1.         -2.11822553] 0.00074676
06 [ 1.         -2.28726564] 0.00026660
07 [ 1.         -2.44674695] 0.00009550
08 [ 1.         -2.59818489] 0.00003431
09 [ 1.        -2.7427259] 0.00001235
10 [ 1.         -2.88126331] 0.00000445
11 [ 1.         -3.01450997] 0.00000161
12 [ 1.         -3.14304593] 0.00000058
13 [ 1.         -3.26735101] 0.00000021
14 [ 1.         -3.38782765] 0.00000008
